In [ ]:
## Import relevant python packages
import matplotlib.pyplot as plt
import numpy as np
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature
import xarray as xr
import matplotlib.path as mpath
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
import random
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import csv
import pickle
import json
import re
from scipy.signal import windows, convolve

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# helper functions

In [ ]:
# great-circle central angle (haversine); result in degrees
def angular_distance_deg(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2*np.arcsin(np.minimum(1.0, np.sqrt(a)))
    return np.degrees(c)

In [ ]:

def read_csv_with_metadata(path):
    """
    Reads a CSV whose first lines are metadata starting with '#',
    followed by a header row and data.

    Returns
    -------
    df : pandas.DataFrame
        The data table.
    meta : dict
        Parsed metadata (e.g., {'Lat': -34.666, 'Cruise': 'ABC123'}).
    """
    meta = {}

    # --- First pass: parse the metadata lines (those starting with '#') ---
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            s = line.lstrip()
            if not s.startswith("#"):
                # first non-comment line should be the header; stop scanning
                break

            # Parse lines like "# Lat = -34.666" or "#Lat: -34.666"
            m = re.match(r"^\s*#\s*([^=:]+)\s*[:=]\s*(.*)\s*$", s)
            if not m:
                continue
            key = m.group(1).strip()
            val = m.group(2).strip().strip("'\"")

            # Try to coerce to number/bool if possible
            num = pd.to_numeric(val, errors="coerce")
            if pd.notna(num):
                # If it parses as a number, store as float (or int if exact)
                meta[key] = int(num) if float(num).is_integer() else float(num)
            else:
                low = val.lower()
                if low in ("true", "false"):
                    meta[key] = (low == "true")
                else:
                    meta[key] = val

    # --- Second pass: load the table (pandas ignores '#' lines via comment) ---
    df = pd.read_csv(path, comment="#")

    # Optional: carry metadata along with the DataFrame
    df.attrs.update(meta)

    return df, meta

# -----------------------
# Example usage
# -----------------------
# path = "your_file.csv"
# df, meta = read_csv_with_metadata(path)
# print(df.head())
# print(meta)

# Access numeric metadata for calculations:
# lat = float(meta.get("Lat"))           # e.g., -34.666
# lon = float(meta.get("Lon", meta.get("Longitude", float("nan"))))


In [ ]:

# Define the predict_SP function
def predict_SP(t, p, gmm, train_data, samples=3000):
    if isinstance(train_data, pd.DataFrame):
        potential_SP = np.linspace(train_data['salinity'].min(), train_data['salinity'].max(), samples)
    else:
        raise ValueError("train_data should be a pandas DataFrame")
    
    tp_sp_combinations = pd.DataFrame({
        'temp': [t] * samples,
        'pressure': [p] * samples,
        'salinity': potential_SP
    })
        
    #tp_sp_combinations = np.array([[t, p, sp] for sp in potential_SP])
    log_likelihoods = gmm.score_samples(tp_sp_combinations)
    best_sp = potential_SP[np.argmax(log_likelihoods)]
    
    return best_sp

def predict_SP_proba(t, p, gmm, train_data, samples=3000):
    if isinstance(train_data, pd.DataFrame):
        potential_SP = np.linspace(train_data['salinity'].min(), train_data['salinity'].max(), samples)
    else:
        raise ValueError("train_data should be a pandas DataFrame")
    
    tp_sp_combinations = pd.DataFrame({
        'temp': [t] * samples,
        'pressure': [p] * samples,
        'salinity': potential_SP
    })
        
    proba = gmm.predict_proba(tp_sp_combinations)
    
    return proba

# set details

In [ ]:
Leg  = 'DG2401_L4_NCT'
Station = 'NC4_OM11_4500'
# Latitude and longitude tolerance
deg_tolerance = 5
year_tolerance = 10
p_min = 1500
##### these paths will need to be changed to run on another computer ####
base_path_raw = '/Users/00096315/Library/CloudStorage/OneDrive-UWA/EXT-MUDSRC - Documents/Cruises/2024_02_Nova Canton/Data/CTD/NC4/Landers/Processed/2_dbar_avg'

## file locations

In [ ]:
## Define file paths and variables
##### these paths will need to be changed to run on another computer ####
base_path_argo = '/Users/00096315/Library/CloudStorage/OneDrive-UWA/01 Work/Working Nate/Summer Project/Data/Publically_available_data/Argo'
base_path_cchdo = '/Users/00096315/Library/CloudStorage/OneDrive-UWA/01 Work/Working Nate/Summer Project/Data/Publically_available_data/CCHDO'
base_path_good_dat = '/Users/00096315/Library/CloudStorage/OneDrive-UWA/01 Work/Working Nate/Summer Project/Data/Collected_data/Pressure_temp_salinity'

figure_file = '/Users/00096315/Library/CloudStorage/OneDrive-UWA/01 Work/03. Python/Projects/NOV/ML_profiles/gmm_figs'
folder_path_out = '/Users/00096315/Library/CloudStorage/OneDrive-UWA/01 Work/03. Python/Projects/NOV/ML_profiles/gmm_out'
folder_matched_files_out = '/Users/00096315/Library/CloudStorage/OneDrive-UWA/01 Work/03. Python/Projects/NOV/ML_profiles/gmm_out/matched_profiles_details/'
#filepath_out_btm = '/Users/jesskolbusz/Library/CloudStorage/OneDrive-SharedLibraries-TheUniversityofWesternAustralia/EXT-MUDSRC - Documents/PROJECTS/2024_JapanTrench/Data/Env/CTD/Processed/GMM/Output/Bottom'
results_df = pd.DataFrame(columns=['Station', 'Number of Clusters', 'Type of Covariance', 'RMSE', 'Means', 'Covariances', 'Weights'])

summary_argo_csv = '/Users/00096315/Library/CloudStorage/OneDrive-UWA/01 Work/03. Python/Projects/NOV/ML_profiles/argo/NOV_profiles_summary_over2500.csv'
summary_cchdo_csv = '/Users/00096315/Library/CloudStorage/OneDrive-UWA/01 Work/03. Python/Projects/NOV/ML_profiles/NOV_profiles_cchdo_summary_over2500.csv'

df_argo = pd.read_csv(summary_argo_csv)
df_argo["time"] = pd.to_datetime(df_argo["time"], errors="coerce")
df_argo["year"] = df_argo["time"].dt.year

df_cchdo = pd.read_csv(summary_cchdo_csv)
df_cchdo["time"] = pd.to_datetime(df_cchdo["time"], errors="coerce")
df_cchdo["year"] = df_cchdo["time"].dt.year

dep_log_file = '/Users/00096315/Library/CloudStorage/OneDrive-UWA/01 Work/03. Python/Projects/NOV/ML_profiles/2024_02_Nova_Canton_Oceanographic_Data.xlsx'
dep_log = pd.read_excel(dep_log_file)


In [ ]:
filtered_dep_log = dep_log[dep_log['Station'] == Station]

target_lat = filtered_dep_log['Lat'].iloc[0]
target_lon = filtered_dep_log['Lon'].iloc[0]
target_year = filtered_dep_log['Date'].iloc[0].year  # <- set this to your reference date

file_pred = os.path.join(base_path_raw, Station + '_processed_pub.csv')

In [ ]:
### get files within bounds for argo
ang = angular_distance_deg(df_argo["lat"].to_numpy(), df_argo["lon"].to_numpy(), target_lat, target_lon)
mask = ang <= deg_tolerance
# years difference (approx using 365.25)
year_diff = (df_argo["year"] - target_year).abs()
mask_time = year_diff <= year_tolerance

### get files within bounds for cchdo
mask_cchdo = mask & mask_time
matched_files1 = df_argo.loc[mask_cchdo, "profile_csv"].dropna().unique()
ang_cchdo = angular_distance_deg(df_cchdo["lat"].to_numpy(), df_cchdo["lon"].to_numpy(), target_lat, target_lon)
mask_cchdo = ang_cchdo <= deg_tolerance
# years difference (approx using 365.25)
year_diff = (df_cchdo["year"] - target_year).abs()
mask_time_cchdo = year_diff <= year_tolerance
mask_cchdo = mask_cchdo & mask_time_cchdo
matched_files2 = df_cchdo.loc[mask_cchdo, "profile_csv"].dropna().unique()

## all files together
all_files_matched = np.concatenate([matched_files1, matched_files2])

### Save matched filenames to CSV 
pd.DataFrame({"file": all_files_matched}).to_csv(os.path.join(folder_matched_files_out, "matched_filenames_" + Station + ".csv"), index=False)
print(f"{len(all_files_matched)} files saved to matched_filenames_{Station}.csv")


In [ ]:
## load all the matched files 
dfs = (pd.read_csv(f) for f in all_files_matched)

filtered_dfs = (
    df[(df['pressure'] >= p_min) &(df['pressure_qc'] <= 2) & (df['temp_qc'] <= 2) & (df['salinity_qc'] <= 2)]
    for df in dfs
)

# GMM run

In [ ]:
## GMM settings
randstat = 42
cov_type= 'full' #tied, diag, spherical 
tsize=0.2

In [ ]:
training_data = pd.concat(filtered_dfs, ignore_index=True)
training_data.dropna(subset=['temp', 'pressure', 'salinity'], inplace=True)

X_train = training_data[['temp', 'pressure', 'salinity']].values

In [ ]:

## Initialise empty lists for varying numbers of clusters and 
## labels, AIC/BIc, etc. 

n_clusters = []
labels_gmm = []
centers_gmm = []
aic_gmm = []
bic_gmm = []

k_clusters = range(1,15)

for k in tqdm(k_clusters):
    n_clusters.append(k)
    ## Define the GMM model
    gmm = GaussianMixture(n_components=k, covariance_type= cov_type, random_state = randstat)
    # Fit and predict the GMM to the control data
    labels_gmm.append(gmm.fit_predict(np.array(X_train)))
    centers_gmm.append(gmm.means_)
    aic_gmm.append(gmm.aic(X_train))
    bic_gmm.append(gmm.bic(X_train))

In [ ]:
##### elbow method input where changes #####
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))
plt.subplots_adjust(hspace = 0.3, wspace=0.3)

axs.plot(n_clusters, np.array(bic_gmm)/10**6, 'r-o', label='BIC')
axs.plot(n_clusters, np.array(aic_gmm)/10**6, 'b-o', label='AIC')
axs.scatter(n_clusters[np.argmin(aic_gmm)], aic_gmm[np.argmin(aic_gmm)]/10**6, 50, color='blue', ec='k', zorder=10000)
axs.scatter(n_clusters[np.argmin(bic_gmm)], bic_gmm[np.argmin(bic_gmm)]/10**6, 50, color='red', ec='k', zorder=10000)

axs.set_xticks(n_clusters)
axs.set_xlabel('Number of clusters')
axs.set_xlim(1,14)
#axs.set_ylim(-5.5,-3)

axs.grid()
axs.set_ylabel(r'AIC/BIC [$\times10^6$]')
axs.legend()

plt.suptitle('Determination of optimal number of clusters', fontsize=12)
plt.savefig(os.path.join(figure_file, Station + '_Scores_clusters_' + cov_type + '.png'), dpi=300, bbox_inches='tight')

plt.show()

## input n clusters based on elbow

In [ ]:
#### USER INPUT VALUES ####
n_components = 8# Example number of components

In [ ]:
# Split the training data into training and evaluation sets - use 20% for evaluation
train_data, eval_data = train_test_split(X_train, test_size=tsize, random_state=randstat)

# Ensure they are DataFrames
train_data = pd.DataFrame(train_data)
train_data.columns = ['temp','pressure','salinity']

eval_data = pd.DataFrame(eval_data)
eval_data.columns = ['temp','pressure','salinity']


gmm = GaussianMixture(n_components=n_components, covariance_type= cov_type, random_state = randstat)
gmm.fit(train_data)

## predict

In [ ]:
# Assuming profile_data is a pandas DataFrame with columns 'Temp' and 'Pressure'
## predict and get 
df, meta = read_csv_with_metadata(file_pred)

df_new = df[['p','tnc90C']] 
df_new.columns = ['pressure','temp']
df_new = df_new[(df_new['pressure'] >= 1500)]

profile_data = df_new[['pressure','temp']]
profile_data = profile_data.reset_index(drop=True)

# Sample temperature and pressure data
temperature = np.array(df_new['temp'])
pressure = np.array(df_new['pressure'])

# Stack temperature and pressure arrays column-wise
# Initialize list to store predicted salinity values
predicted_sps_profile = []
predicted_sp_proba_prof = []

# Iterate over the profile data to predict salinity
for index, row in profile_data.iterrows():
    t_real = row['temp']
    p_real = row['pressure']
    
    # Predict salinity using the predict_SP function
    predicted_sp = predict_SP(t_real, p_real, gmm, train_data)
    predicted_sp_proba = predict_SP_proba(t_real, p_real, gmm, train_data)

    # Append predicted salinity value to the list
    predicted_sps_profile.append(predicted_sp)
    predicted_sp_proba_prof.append(predicted_sp_proba)


# If you have actual salinity values for profile_data, you can calculate RMSE here
# rmse_profile = np.sqrt(mean_squared_error(actual_sps_profile, predicted_sps_profile))
# print(f"RMSE for profile data: {rmse_profile}")

profile_data['Pred_SP'] = predicted_sps_profile

df_new['Predicted_SP'] = predicted_sps_profile
df_new.to_csv(os.path.join(folder_path_out, Station + '_modelled.csv'), index=False)

profile_data.head()

## plot data

In [ ]:
import gsw
df_plot = df[['p','tnc90C','SP']] # Salinity_PSU #Conductivity_mScm
#df_plot['Sal'] = gsw.SP_from_C(df_plot['Conductivity_mScm'], df_plot['Temp'], df_plot['Pressure_dbar'])
pmax = df_plot['p'].max() +200


fig, axs = plt.subplots(1, 2, figsize=(10, 8))

    #axs[0].plot(df_new['SP'],  df_new['depth'])
axs[0].plot(profile_data['Pred_SP'], profile_data['pressure'])
axs[0].plot(df_plot['SP'], df_plot['p'], color = 'red')
axs[0].set_ylim(1000, pmax)
axs[0].invert_yaxis()
axs[0].set_ylabel("Pressure [dbar]")
axs[0].set_xlabel('salinity (PSU)')
axs[0].set_xlabel('SP')
axs[0].grid(True)


axs[1].plot(profile_data['temp'], profile_data['pressure'])
axs[1].set_ylim(1000, pmax)
axs[1].invert_yaxis()
axs[1].set_ylabel("Pressure [dbar]")
axs[1].set_xlabel('Temperature (°C)')
axs[1].grid(True)

    #figure_file = '/Users/jesskolbusz/Desktop/Full_data/figs/'

fig.align_labels()
plt.tight_layout()
plt.show()

fig.savefig(os.path.join(figure_file, Station + '_modelled_' + cov_type + '.png'), dpi=300)
    


## evaluation details out

In [ ]:
# Initialize lists to store actual and predicted salinity values
actual_sps_eval = []
predicted_sps_eval = []

# Iterate over the evaluation data to predict salinity
for index, row in eval_data.iterrows():
    t_real = row['temp']
    p_real = row['pressure']
    actual_sp = row['salinity']
    
    # Predict salinity using the predict_SP function
    predicted_sp = predict_SP(t_real, p_real, gmm, train_data)
    
    # Append actual and predicted salinity values to lists
    actual_sps_eval.append(actual_sp)
    predicted_sps_eval.append(predicted_sp)

# Calculate RMSE for the evaluation set
rmse_eval = np.sqrt(mean_squared_error(actual_sps_eval, predicted_sps_eval))
print(f"RMSE for evaluation data: {rmse_eval}")

In [ ]:
# Extract GMM information and store in variables
means = gmm.means_
covariances = gmm.covariances_
weights = gmm.weights_

# Prepare the GMM info as strings to store in the DataFrame
means_str = json.dumps(means.tolist())
covariances_str = json.dumps(covariances.tolist())
weights_str = json.dumps(weights.tolist())

# Append the new results to the results DataFrame
new_row = pd.DataFrame([{
    'Station': Station,
    'Type of Covariance': cov_type,
    'Number of Clusters': n_components,
    'RMSE': rmse_eval,
    'Means': means_str,
    'Covariances': covariances_str,
    'Weights': weights_str
}])

###### Append the new results to the results DataFramed
#### DOESNT CONCAT PROPERLY???? ###
results_df = pd.concat([results_df, new_row], ignore_index=True)
results_df.to_csv(os.path.join(folder_path_out, 'Results_means.csv'), index=False)


#   Plot all out modelled profiles 

In [ ]:
from scipy.signal import windows, convolve
# ---- helper: NaN-safe convolution along 1D ----
def smooth_nan_safe(y, win_len=11):
    y = np.asarray(y, dtype=float)
    w = windows.hann(win_len)
    w = w / w.sum()
    valid = np.isfinite(y).astype(float)
    y_filled = np.where(np.isfinite(y), y, 0.0)
    num = convolve(y_filled, w, mode='same')
    den = convolve(valid,   w, mode='same')
    out = num / np.where(den > 1e-12, den, np.nan)
    return out

In [ ]:

all_profiles = []
path_start = '/Users/00096315/Library/CloudStorage/OneDrive-UWA/01 Work/03. Python/Projects/NOV/ML_profiles/gmm_out'
win_len = 11  # ~20 dbar at 2 dbar spacing

for files in os.listdir(path_start):
    if files.endswith('_modelled.csv'):
        fpath = os.path.join(path_start, files)
        try:
            df = pd.read_csv(fpath)

            Station = fpath.split('/')[-1].replace('_modelled.csv', '')  # Add station name
            df['Station']  = Station
            filtered_dep_log = dep_log[dep_log['Station'] == Station]

            target_lat = filtered_dep_log['Lat'].iloc[0]
            target_lon = filtered_dep_log['Lon'].iloc[0]

            win = windows.hann(11)
            df['t_smooth']  = smooth_nan_safe(df['temp'].values, win_len)

            df['SA'] = gsw.SA_from_SP(df['Predicted_SP'].values, df['pressure'].values, target_lon, target_lat)  # Replace lon=0, lat=0 with your actual coordinates
            df['CT'] = gsw.CT_from_t(df['SA'].values, df['t_smooth'].values, df['pressure'].values)

            all_profiles.append(df[['SA', 'CT', 'pressure', 'Station']])
        except Exception as e:
            print(f"Error loading {fpath}: {e}")

In [ ]:
# Convert to DataFrame
plot_df = pd.concat(all_profiles, ignore_index=True)

# (Optional) drop rows with missing SA/CT
plot_df = plot_df.dropna(subset=['SA', 'CT'])

# Plot with Plotly (use scattergl if very large)
fig = px.scatter(
    plot_df,
    x='SA',
    y='CT',
    color='Station',
    hover_data={'Station': True, 'SA': ':.3f', 'CT': ':.3f'},
    title='CTD Profiles —  Nova Canton Leg 2',
    render_mode='webgl'  # faster for many points
)

fig.update_layout(
    xaxis=dict(title='Absolute Salinity (g/kg)'), #, range=[33.5, 35.0]),
    yaxis=dict(title='Conservative Temperature (°C)'), #, range=[-1.5, 2.5]),
    legend_title='Station'
)

pio.renderers.default = 'browser'
fig.show()